# Необходимо решить две задачи:

## 1.Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

## 2.Определить, с каких источников и в какое время суток чаще всего заходят роботы.

### Задача №1 

Опредедение количества ботов и их доли на сайте

In [1]:
#Приступим к решению задачи с формулировки параметров запроса к API "Яндекс Метрики"
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions': 'ym:s:lastsignTrafficSource,ym:s:isRobot',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
#Зададим токен приложения
token = 'вставьте ваш токен'

#И создадим словарь headers, который будет содержать параметры заголовков HTTP
headers = {'Authorization': 'OAuth ' + token}

In [2]:
#Далее загрузим все библиотеки, необходимые для комфортной работы с отчетом
import json 
import requests
from pprint import pprint

In [3]:
#Сформулируем запрос и проверим, верен ли он
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


In [4]:
#Переведем ответ в формат json и созадим для него специальную переменную
metrika_data = response.json()

In [5]:
#Проверим, к какому типу данных относится полученная нами переменная
print(type(metrika_data))

<class 'dict'>


In [6]:
#Мы получили словарь, теперь нужно посмотреть, какие в нем имеются ключи
print(metrika_data.keys())

dict_keys(['query', 'data', 'total_rows', 'total_rows_rounded', 'sampled', 'contains_sensitive_data', 'sample_share', 'sample_size', 'sample_space', 'data_lag', 'totals', 'min', 'max'])


In [7]:
#Также узнаем, сколько соответствующих запросу строк находится в этом словаре
metrika_data['total_rows']

13

In [8]:
#Теперь с помощью pprint посмотрим, какие данные находятся в словаре по ключу data,
#строк всего 13, поэтому можно вывести их все
pprint(metrika_data['data'][0:13])

[{'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': 'no', 'name': 'People'}],
  'metrics': [2595.0, 0.0]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': 'yes', 'name': 'Robots'}],
  'metrics': [739.0, 100.0]},
 {'dimensions': [{'icon_id': '0',
                  'icon_type': 'traffic-source',
                  'id': 'direct',
                  'name': 'Direct traffic'},
                 {'id': 'no', 'name': 'People'}],
  'metrics': [580.0, 0.0]},
 {'dimensions': [{'icon_id': '0',
                  'icon_type': 'traffic-source',
                  'id': 'direct',
                  'name': 'Direct traffic'},
                 {'id': 'yes', 'name': 'Robots'}],
  'metrics': [177.0, 100.0]},
 {'dimensions': [

In [9]:
#Далее потребуется преобразовать имеющийся спикок словарей в датафрейм, для чего необходимо загрузить библиотеку pandas
import pandas as pd 

In [10]:
#Создадим датафрейм на основе списка
metrika_df = pd.DataFrame(metrika_data['data'])
display(metrika_df.head(13))

,dimensions,metrics
0,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[2595.0, 0.0]"
1,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[739.0, 100.0]"
2,"[{'icon_id': '0', 'icon_type': 'traffic-source...","[580.0, 0.0]"
3,"[{'icon_id': '0', 'icon_type': 'traffic-source...","[177.0, 100.0]"
4,"[{'icon_id': '1', 'icon_type': 'traffic-source...","[95.0, 0.0]"
5,"[{'icon_id': '8', 'icon_type': 'traffic-source...","[69.0, 0.0]"
6,"[{'icon_id': '1', 'icon_type': 'traffic-source...","[31.0, 100.0]"
7,"[{'icon_id': '8', 'icon_type': 'traffic-source...","[11.0, 100.0]"
8,"[{'icon_id': '-1', 'icon_type': 'traffic-sourc...","[7.0, 0.0]"
9,"[{'icon_id': '4', 'icon_type': 'traffic-source...","[5.0, 0.0]"


In [11]:
#Используем функцию для преобразования данных в плоский вид
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

In [12]:
#И применим ее, посмотрев на результат
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:5])

[{'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 0.0,
  'ym:s:visits': 2595.0},
 {'ym:s:isRobot': 'Robots',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 100.0,
  'ym:s:visits': 739.0},
 {'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:robotPercentage': 0.0,
  'ym:s:visits': 580.0},
 {'ym:s:isRobot': 'Robots',
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:robotPercentage': 100.0,
  'ym:s:visits': 177.0},
 {'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Link traffic',
  'ym:s:robotPercentage': 0.0,
  'ym:s:visits': 95.0}]


In [13]:
#Создадим новый датафрейм из списка
metrika_df = pd.DataFrame(metrika_list_of_dicts)
display(metrika_df.head())

,ym:s:lastsignTrafficSource,ym:s:isRobot,ym:s:visits,ym:s:robotPercentage
0,Search engine traffic,People,2595.0,0.0
1,Search engine traffic,Robots,739.0,100.0
2,Direct traffic,People,580.0,0.0
3,Direct traffic,Robots,177.0,100.0
4,Link traffic,People,95.0,0.0


In [14]:
#Теперь преобразуем его в сводную таблицу, заодно проверив данные на пропуски и выведя суммуарные значения
pivot = metrika_df.pivot_table( index = ['ym:s:lastsignTrafficSource'],
                      values = ['ym:s:visits'], columns = ['ym:s:isRobot'],
                      aggfunc = 'sum',
                      margins = True,
                      fill_value=0)
pivot.head(7)

ym:s:visits               
ym:s:isRobot                       People Robots     All
ym:s:lastsignTrafficSource                              
Cached page traffic                     5      1     6.0
Direct traffic                        580    177   757.0
Internal traffic                        7      3    10.0
Link traffic                           95     31   126.0
Recommendation system traffic           2      0     2.0
Search engine traffic                2595    739  3334.0
Social network traffic                 69     11    80.0

In [15]:
#Сбросим двойную индескацию и сделаем источник траффика отдельной колонкой
pivot.columns=list(map('_'.join,pivot))
metrika_df = pivot.reset_index()
metrika_df.head(7)

,ym:s:lastsignTrafficSource,ym:s:visits_People,ym:s:visits_Robots,ym:s:visits_All
0,Cached page traffic,5,1,6.0
1,Direct traffic,580,177,757.0
2,Internal traffic,7,3,10.0
3,Link traffic,95,31,126.0
4,Recommendation system traffic,2,0,2.0
5,Search engine traffic,2595,739,3334.0
6,Social network traffic,69,11,80.0


In [16]:
#Переимениуем колонки для удобства воспрятия
metrika_df.columns=['Source','People','Robots', 'Total']



In [17]:
#Добавим в датафрейм колонку, показывающуу процент роботов среди всех посетителей сайта
metrika_df['Robot Share'] = metrika_df['Robots']/(metrika_df['People']+metrika_df['Robots'])*100
metrika_df['Robot Share'] = metrika_df['Robot Share'].round(1)
display(metrika_df)

,Source,People,Robots,Total,Robot Share
0,Cached page traffic,5,1,6.0,16.7
1,Direct traffic,580,177,757.0,23.4
2,Internal traffic,7,3,10.0,30.0
3,Link traffic,95,31,126.0,24.6
4,Recommendation system traffic,2,0,2.0,0.0
5,Search engine traffic,2595,739,3334.0,22.2
6,Social network traffic,69,11,80.0,13.8
7,All,3353,962,4315.0,22.3


### Вывод №1
Итак, мы выяснили, что из 3353 посетителей сайта в сентябре 2020 года 962 было ботами, что составляет чуть менее четверти от общего числа (22,3%).

На платформе была упомянута гипотеза, но она не была сформулирована, и не было дано задание ее сформулировать самостоятельно, так что что-либо сказать по этому поводу довольно трудно




## Задание  №2
Перейдем ко второй задаче и определим, с каких источников и в какое время суток чаще всего заходят роботы.

In [18]:
#Составим второй запрос к "Яндекс Метрики", используя несколько группировку по часам
params2 = {'metrics': 'ym:s:visits, ym:s:robotPercentage',
          'dimensions': 'ym:s:hour,ym:s:lastsignTrafficSource' ,  
          'date1': '2020-10-07',  
          'date2': '2020-10-09',    
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params2, headers=headers)
print (response.status_code)
#Переведем ответ в формат json и созадим для него специальную переменную
metrika_data2 = response.json()
#Преобразуем эту переменную в новый датайрейм
metrika_df2 = pd.DataFrame(metrika_data2['data'])
#Применим к датафрейму уже использовавшуюся функцию
metrika_list_of_dicts2 = getMetrikaDataInListOfDicts(metrika_data2)
#Создадим новый датафрейм из списка
metrika_df2 = pd.DataFrame(metrika_list_of_dicts2)

display(metrika_df2.head())

200


,ym:s:hour,ym:s:lastsignTrafficSource,ym:s:visits,ym:s:robotPercentage
0,13:00,Search engine traffic,30.0,30.000000
1,14:00,Search engine traffic,25.0,32.000000
2,11:00,Search engine traffic,23.0,26.086957
3,17:00,Search engine traffic,22.0,31.818182
4,22:00,Search engine traffic,22.0,9.090909


In [19]:
#Преобразуем датафрейм в сводную таблицу
pivot2 = metrika_df2.pivot_table( index = ['ym:s:hour'],
                      values = ['ym:s:robotPercentage'], columns = ['ym:s:lastsignTrafficSource'],
                      aggfunc = 'sum',
                      
                      fill_value=0)
pivot2.head()

ym:s:robotPercentage                                \
ym:s:lastsignTrafficSource       Direct traffic Internal traffic Link traffic   
ym:s:hour                                                                       
00:00                                 33.333333                0          0.0   
01:00                                  0.000000                0          0.0   
02:00                                  0.000000                0          0.0   
03:00                                100.000000                0          0.0   
04:00                                  0.000000                0          0.0   

                                                                         
ym:s:lastsignTrafficSource Search engine traffic Social network traffic  
ym:s:hour                                                                
00:00                                  41.666667                      0  
01:00                                   0.000000                      0  
02:00                                   0.000000                      0  
03:00                                   0.000000                      0  
04:00                                   0.000000                      0

In [20]:
#Уберем двойную индексацию и сделаем Hour отдельным стоблцом
pivot2.columns=list(map('_'.join,pivot2))
metrika_df2 = pivot2.reset_index()
metrika_df2.head(7)

,ym:s:hour,ym:s:robotPercentage_Direct traffic,ym:s:robotPercentage_Internal traffic,ym:s:robotPercentage_Link traffic,ym:s:robotPercentage_Search engine traffic,ym:s:robotPercentage_Social network traffic
0,00:00,33.333333,0,0.0,41.666667,0
1,01:00,0.000000,0,0.0,0.000000,0
2,02:00,0.000000,0,0.0,0.000000,0
3,03:00,100.000000,0,0.0,0.000000,0
4,04:00,0.000000,0,0.0,0.000000,0
5,05:00,0.000000,0,0.0,25.000000,0
6,06:00,25.000000,0,0.0,11.111111,0


In [21]:
#Переименуем колонки
metrika_df2.columns=['Hour','Direct tr.','Internal tr.', 'Link tr.','Search Engine tr.','Social Network tr.']
#Округлим числа до одного знака после запятой
metrika_df2.round(1)

,Hour,Direct tr.,Internal tr.,Link tr.,Search Engine tr.,Social Network tr.
0,00:00,33.3,0,0.0,41.7,0
1,01:00,0.0,0,0.0,0.0,0
2,02:00,0.0,0,0.0,0.0,0
3,03:00,100.0,0,0.0,0.0,0
4,04:00,0.0,0,0.0,0.0,0
5,05:00,0.0,0,0.0,25.0,0
6,06:00,25.0,0,0.0,11.1,0
7,07:00,0.0,0,0.0,20.0,0
8,08:00,0.0,0,0.0,16.7,0
9,09:00,28.6,0,33.3,21.4,0


## Вывод №2
Проведенный анализ показал, что наиболее популярными источниками трафика для ботов являются Search Engine и Direct, в то время как остальные источники практически не используются ботами.
Что касается времени визитов, то боты заходят на сайт практически круглосуточно, за ислючением глубокой ночи. В остальном доля их визитов через Search Engine варьируется в течение дня, достигая двух максимумов: в промежутке между 13:00 и 19:00 и с 23:00 до 01:00. Посредством Direct traffic боты появляются на сайте инретвалами, разделенными перерывами длительностью от одного до двух часов в течение суток и с трехачасовым перерывом между 21:00 и полуночью.